In [9]:
# !pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 3.7 MB/s eta 0:00:00


In [106]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from datetime import datetime
import time
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score,confusion_matrix,ConfusionMatrixDisplay,roc_curve,roc_auc_score,precision_recall_curve
from sklearn.ensemble import RandomForestClassifier , StackingClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression,Lasso
from sklearn.preprocessing import Binarizer
from sklearn.model_selection import cross_val_score,GridSearchCV

# 한글 깨짐 방지
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'malgun Gothic'

In [107]:
test = pd.read_csv('/content/drive/MyDrive/Ubion_project_2/test.csv', encoding='euc-kr')
train = pd.read_csv('/content/drive/MyDrive/Ubion_project_2/train_Winsorization.csv', encoding='euc-kr')

In [108]:
train['기업수명주기'] = train['기업수명주기'].map({
    '도입기' : 1,
    '성장기' : 2,
    '성숙기' : 3,
    '수축기' : 4,
    '쇠퇴기' : 5
}).astype('category')

test['기업수명주기'] = test['기업수명주기'].map({
    '도입기' : 1,
    '성장기' : 2,
    '성숙기' : 3,
    '수축기' : 4,
    '쇠퇴기' : 5
}).astype('category')

train['파부비초과여부']=train['파부비초과여부'].astype('category')
test['파부비초과여부']=test['파부비초과여부'].astype('category')
train['파차의초과여부']=train['파차의초과여부'].astype('category')
test['파차의초과여부']=test['파차의초과여부'].astype('category')

In [109]:
# train=train[['총자본증가율',
# '총자산대비잉여현금흐름',
# '당좌자산회전률',
# '순운전자본회전률',
# '자기자본회전률',
# 'log자산총계',
# '자기자본증가율',
# '기업수명주기',
# '총자산대비현금흐름',
# '매출액대비잉여현금흐름',
# '영업이익증가율',
# '총자본투자효율',
# '총자본순이익률',
# '파부비초과여부',
# '파차의초과여부','t-1감사의견코드']]

# test=test[['총자본증가율',
# '총자산대비잉여현금흐름',
# '당좌자산회전률',
# '순운전자본회전률',
# '자기자본회전률',
# 'log자산총계',
# '자기자본증가율',
# '기업수명주기',
# '총자산대비현금흐름',
# '매출액대비잉여현금흐름',
# '영업이익증가율',
# '총자본투자효율',
# '총자본순이익률',
# '파부비초과여부',
# '파차의초과여부','t-1감사의견코드']]

In [110]:
X_train_int=train[['총자본증가율',
'총자산대비잉여현금흐름',
'당좌자산회전률',
'순운전자본회전률',
'자기자본회전률',
'log자산총계',
'자기자본증가율',
'총자산대비현금흐름',
'매출액대비잉여현금흐름',
'영업이익증가율',
'총자본투자효율',
'총자본순이익률',
't-1감사의견코드']]
X_train_cat=train[['기업수명주기','파부비초과여부','파차의초과여부']]

X_test_int=test[['총자본증가율',
'총자산대비잉여현금흐름',
'당좌자산회전률',
'순운전자본회전률',
'자기자본회전률',
'log자산총계',
'자기자본증가율',
'총자산대비현금흐름',
'매출액대비잉여현금흐름',
'영업이익증가율',
'총자본투자효율',
'총자본순이익률',
't-1감사의견코드']]
X_test_cat=test[['기업수명주기','파부비초과여부','파차의초과여부']]

In [111]:
X_train = train.drop('t-1감사의견코드',axis=1)
y_train = train[['t-1감사의견코드']]

X_test = test.drop('t-1감사의견코드',axis=1)
y_test = test[['t-1감사의견코드']]

In [112]:
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train_int)
X_test_sc = scaler.transform(X_test_int)

In [113]:
X_train_sc=pd.DataFrame(X_train_sc, columns=X_train_int.columns).drop('t-1감사의견코드',axis=1)
X_test_sc=pd.DataFrame(X_test_sc, columns=X_test_int.columns).drop('t-1감사의견코드',axis=1)

In [114]:
X_train_sum = pd.concat([X_train_sc,X_train_cat],axis=1)
X_test_sum = pd.concat([X_test_sc,X_test_cat],axis=1)

---

In [115]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from imblearn.under_sampling import TomekLinks

def model_with_tomek_links(x_train, y_train, x_test, y_test):
    categorical_features_indices = [12, 13, 14]

    models = [
        LogisticRegression(C=0.01, penalty='l2', random_state=0),
        RandomForestClassifier(random_state=0),
        LGBMClassifier(random_state=0),
        CatBoostClassifier(random_state=0, cat_features=categorical_features_indices, verbose=False),
        SVC(random_state=0)
    ]

    rdict = {
        'model': [],
        'sampling_ratio': [],
        'acc_train': [],
        'auc_train': [],
        'acc_test': [],
        'precision': [],
        'recall': [],
        'f1_score': [],
        'AUC_test': []
    }

    # Tomek Links를 사용하여 언더샘플링된 데이터 생성
    tl = TomekLinks()
    x_train_undersampled, y_train_undersampled = tl.fit_resample(x_train, y_train)

    for clf in models:
        # Train
        clf = clf.fit(x_train_undersampled, y_train_undersampled)
        y_hat_train = clf.predict(x_train_undersampled)
        results_train = (round(accuracy_score(y_train_undersampled, y_hat_train), 4), round(roc_auc_score(y_train_undersampled, y_hat_train), 4))

        # Test
        y_hat_test = clf.predict(x_test)
        results = (round(accuracy_score(y_test, y_hat_test), 4),
                   round(precision_score(y_test, y_hat_test), 4),
                   round(recall_score(y_test, y_hat_test), 4),
                   round(f1_score(y_test, y_hat_test), 4),
                   round(roc_auc_score(y_test, y_hat_test), 4))

        rdict['model'].append(type(clf).__name__)
        rdict['sampling_ratio'].append('Tomek Links')
        rdict['acc_train'].append(results_train[0])
        rdict['auc_train'].append(results_train[1])
        rdict['acc_test'].append(results[0])
        rdict['precision'].append(results[1])
        rdict['recall'].append(results[2])
        rdict['f1_score'].append(results[3])
        rdict['AUC_test'].append(results[4])

        confusion = confusion_matrix(y_test, y_hat_test)
        print(f"Model: {type(clf).__name__}, Sampling Ratio: Tomek Links")
        print(confusion)

    rdf_final = pd.DataFrame(data=rdict)
    return rdf_final

In [116]:
model_with_tomek_links(X_train_sum, y_train, X_test_sum, y_test)

Model: LogisticRegression, Sampling Ratio: Tomek Links
[[37679   578]
 [ 2495   373]]
Model: RandomForestClassifier, Sampling Ratio: Tomek Links
[[37364   893]
 [ 1463  1405]]
Model: LGBMClassifier, Sampling Ratio: Tomek Links
[[37219  1038]
 [ 1358  1510]]
Model: CatBoostClassifier, Sampling Ratio: Tomek Links
[[37254  1003]
 [ 1365  1503]]
Model: SVC, Sampling Ratio: Tomek Links
[[37790   467]
 [ 2127   741]]


,model,sampling_ratio,acc_train,auc_train,acc_test,precision,recall,f1_score,AUC_test
0,LogisticRegression,Tomek Links,0.9277,0.5576,0.9253,0.3922,0.1301,0.1953,0.5575
1,RandomForestClassifier,Tomek Links,1.0000,0.9998,0.9427,0.6114,0.4899,0.5439,0.7333
2,LGBMClassifier,Tomek Links,0.9519,0.7745,0.9417,0.5926,0.5265,0.5576,0.7497
3,CatBoostClassifier,Tomek Links,0.9620,0.8152,0.9424,0.5998,0.5241,0.5594,0.7489
4,SVC,Tomek Links,0.9401,0.6383,0.9369,0.6134,0.2584,0.3636,0.6231
